# Analysis for Corona Check dataset

## Purpose of this Notebook
- [x] Get an statistical overview
    - [x] How many users?
    - [x] How many assessments (= filled out questionnaires)?
    - [x] Date range of the dataset?
    - [x] User-assessment distribution
    - [x] Baseline statistics

## Preparation
Import modules and load data for later use

In [1]:
 import math as m
import numpy as np
import pandas as pd
from datetime import date, datetime

In [2]:
# to import own modules, a workaround - assuming the curent working directory is the folder (your/local/path/UsAs/src/d01_analysis)
import sys
# if this code does not work for you, point the path to the src folder
sys.path.insert(0, "../..")

from src.d00_utils import cc_helpers, helpers

In [3]:
# read in dataframe
# assuming current working directory is the folder (your/local/path/UsAs/src/d01_analysis)
df = pd.read_csv('../../data/d01_raw/cc/22-10-27_corona-check-data.csv')
df_baseline = pd.read_csv('../../data/d01_raw/cc/22-10-27_corona-check-data-baseline.csv')

### <font color='red'>Problem with the user_id</font>
A user_id does not refer to one person in this questionnaire, since the baseline and followup questionnaire are within ONE questionnaire. That is, we have to make assumptions when a user_id refers to one specific person.
These assumptions are: 
- Do you fill out this questionnaire for yourself? == `YES`
- `Age` must not vary
    - If `Age` varies within the `Author==YES` filtered answers, we take the mode age and drop other assessments.

<font color='red'>This code takes a while to run</font>

In [4]:
print('No of assessments at start:\t', df.shape[0])
df = cc_helpers.drop_one_time_users(df)
print('No of assessments without one time users:\t', df.shape[0])
df = cc_helpers.drop_ambiguous_users(df)
print('No of assessments without ambigious users:\t', df.shape[0])

No of assessments at start:	 89659
No of assessments without one time users:	 50223
No of assessments without ambigious users:	 43993


## Statistical overview
Here we calculate statistics for both the dataset overall and the baseline questionaires

In [5]:
# create result dict to save to disk
result = dict()

### Number of users

In [6]:
# Number of unique users
print('Number of user:\t', df_baseline.user_id.nunique())
result['n_users'] = df_baseline.user_id.nunique()

Number of user:	 13763


### Number of assessments

In [7]:
# Number of unique answers
print('Number of assessments:\t', df.answer_id.nunique())
result['n_assessments'] = df.answer_id.nunique()

Number of assessments:	 43993


### Date Range of assessments

In [8]:
# date range
format = '%Y-%m-%d %H:%M:%S'
date_start = df.created_at.min()
result['First assessment from'] = date_start
date_start = datetime.strptime(date_start, format)
date_end = df.created_at.max()
result['Latest assessment from'] = date_end
date_end = datetime.strptime(date_end, format)

print('Start:\t', date_start)
print('End:\t', date_end)

delta = date_end.date()-date_start.date()

print('\nDate Range in')
print('Years:\t', delta.days/365)
print('Months:\t', delta.days/12)
print('Days:\t', delta.days)

result['Time range in days'] = delta.days

Start:	 2020-04-08 13:48:43
End:	 2022-09-30 14:25:32

Date Range in
Years:	 2.4794520547945207
Months:	 75.41666666666667
Days:	 905


### Distribution of filled out questionnaires

In [9]:
bins = [0, 1, 2, 3, 5, 10, 100, 1000]
ser = pd.cut(df.user_id.value_counts(), bins = bins).value_counts().sort_index()
print(ser)
result.update(dict(ser))

(0, 1]         2018
(1, 2]         7571
(2, 3]         2061
(3, 5]         1285
(5, 10]         572
(10, 100]       236
(100, 1000]      20
Name: user_id, dtype: int64


### Average period length between two filled out questionnaires

In [10]:
# res = result
res = helpers.find_schedule_pattern(df, form='%Y-%m-%d %H:%M:%S', date_col_name='created_at')
result.update(res)

### Age Distribution

Age is recorded in groups with stepsize of 10 (eg. from ages 20 to 29). To be able to work with the data we assume that each user in a given age groug (eg. 20-29) is aged equal to the mean of the group (25). For user in the age group 80+ we assume an age of 85.

The following table shows the number of users in each age group.

In [22]:
user_ages = df_baseline.age.replace(to_replace = {'00-09': 5.5,
                                                  '10-19': 14.5,
                                                  '20-29': 24.5,
                                                  '30-39': 35.5,
                                                  '40-49': 45.5,
                                                  '50-59': 55.5,
                                                  '60-69': 65.5,
                                                  '70-79': 75.5,
                                                  '80+':   85.5}
                                   )

result['user_age_mean'] = user_ages.mean()

Next we calculate the mean age and the standard deviation

In [24]:
result['user_age_standard_deviation'] = user_ages.std()

### Sex Distribution

We calculate the number of users that self identified their sex as male, female or diverse or no answer

In [25]:
df_sex = pd.DataFrame(df_baseline.groupby('gender')['user_id'].count())

result['n_users_male'] = df_sex['user_id']['MALE']
result['n_users_female'] = df_sex['user_id']['FEMALE']
result['n_users_diverse'] = df_sex['user_id']['DIVERSE']
result['n_users_no_answer'] = df_sex['user_id']['NOANSWER']

print(df_sex)

          user_id
gender           
DIVERSE        73
FEMALE       5362
MALE         8042
NOANSWER      286


### Country Statstics
We calculate how many users participated by country

In [26]:
country_series = df_baseline.groupby('country_code')['user_id'].count()
country_series

country_code
AE      61
AF       2
AG       1
AL       4
AO       1
      ... 
VN       1
XK       6
ZA    1346
ZM      13
ZW       9
Name: user_id, Length: 109, dtype: int64

We also calculate the percentage of german-based users in the dataset

In [27]:
result['n_users_german'] = country_series['DE']
result['n_users_non_german'] = country_series.sum()
print('{:.2f}% german-based users in dataset'.format(country_series['DE'] / country_series.sum() * 100))

36.81% german-based users in dataset


### Statistical Overview

In [28]:
result

{'n_users': 13763,
 'n_assessments': 43993,
 'First assessment from': '2020-04-08 13:48:43',
 'Latest assessment from': '2022-09-30 14:25:32',
 'Time range in days': 905,
 Interval(0, 1, closed='right'): 2018,
 Interval(1, 2, closed='right'): 7571,
 Interval(2, 3, closed='right'): 2061,
 Interval(3, 5, closed='right'): 1285,
 Interval(5, 10, closed='right'): 572,
 Interval(10, 100, closed='right'): 236,
 Interval(100, 1000, closed='right'): 20,
 'avg hours between two assessments': 232.35601079339853,
 'avg days between two assessments': 9.681500449724936,
 'std_hours': 614.6605704829816,
 'std_days': 25.61085710345757,
 'user_age_mean': 32.671582760405116,
 'user_age_standard_deviation': 18.007270034497324,
 'n_users_male': 8042,
 'n_users_female': 5362,
 'n_users_diverse': 73,
 'n_users_no_answer': 286,
 'n_users_german': 3326,
 'n_users_non_german': 9035}